<a href="https://colab.research.google.com/github/jaeohshin/pytorch/blob/main/01_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch workflow

Resources:
* https://www.learnpytorch.io/

In [ ]:
what_were_covering = {1: "data (prepare and load")
                      2: "build model"
                      3: "fitting the model to data (training)"
                      4: "making predictions and evaluating a model (inference)"
                      5: "f"
    
}